In [21]:
#Version 0.0 ---comment preprocessing

import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage as nd
import re

Number_of_entry = 5000
#input data and clean up
data = pd.read_csv('aggression_annotated_comments.tsv', sep = '\t', nrows = Number_of_entry)

comment_data = data.comment
comment_data = np.asarray(comment_data)

PUNCTUATION_NO_SPACE = re.compile("[.;:!*`_'?,\"()\[\]]")
PUNCTUATION_SPACE = re.compile("-")
NEWLINE = re.compile("newlinetoken")
#This is more or less the nltk stop list with negations removed
skip = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", \
        "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', \
        'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', \
        'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', \
        "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', \
        'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'if', 'or', \
        'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'between', \
        'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', \
        'out', 'on', 'off', 'over', 'under', 'further', 'then', 'once', 'here', 'there', 'theres','when', 'where',\
        'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',\
        'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', \
        'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y','u','ur']

for i in range(comment_data.size):
    comment_data[i] = PUNCTUATION_NO_SPACE.sub("",comment_data[i].lower())
    comment_data[i] = NEWLINE.sub("",comment_data[i])
    comment_data[i] = PUNCTUATION_SPACE.sub(" ",comment_data[i].lower())
    comment_data[i] = comment_data[i].split()
    comment_data[i] = [word for word in comment_data[i] if word not in skip]
    
print(comment_data[6])


    





#stop_list = nltk.corpus.stopwords.words("english")

['removed', 'followingall', 'names', 'early', 'polish', 'rulers', 'ficticious', 'therefore', 'index', 'naming', 'oda', 'von', 'haldensleben', 'husband', 'dagome', 'records', 'first', 'time', 'rulers', 'polanen', 'tribe', 'therefore', 'indicated', 'first', 'document', 'later', 'developing', 'land', 'named', 'polandthis', 'quite', 'comment', 'names', 'fictitious', 'deserves', 'least', 'backing']


In [30]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

for i in range(comment_data.size):
    lemmatizer = WordNetLemmatizer()
    comment_data[i] = ' '.join([lemmatizer.lemmatize(word) for word in comment_data[i]])
    
print(comment_data[499])

opinion poll conducted mac 2000 45% consider taiwanese four year later think wonder mac either stopped hided opinion poll result would not stimulative prc


[nltk_data] Downloading package wordnet to /Users/Fenggq/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
annotation = pd.read_csv('aggression_annotations.tsv', sep = '\t', nrows =Number_of_entry*10)
target_raw = annotation.aggression
target_pro = np.zeros(Number_of_entry)

ctr = 0
for i in range(target_pro.size):
    aggr = 0
    for j in range(10):
        aggr += target_raw[ctr]
        ctr += 1
    if (aggr/10 > 0.5):
        target_pro[i] = 1
    else:
        target_pro[i] = 0
    
print(target_pro[1])

0.0


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

print(type(comment_data[0]))

comment_data = np.asarray(comment_data)
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(comment_data)
comment_vectorized = tfidf_vectorizer.transform(comment_data)

print(comment_vectorized.shape)

data_train, data_test, target_train, target_test = train_test_split(
    comment_vectorized, target_pro, test_size = 0.1)

model = LinearSVC(C=0.01)
model.fit(data_train, target_train)
accuracy = accuracy_score(target_test, model.predict(data_test))
print("Accuracy: ", accuracy)
       


<class 'str'>
(5000, 27730)
Accuracy:  0.914
